In [1]:
!pip install selenium --upgrade

PRODUCT SEARCH

In [2]:
import selenium
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By

In [3]:
import os
print(os.getcwd())

/Users/sukanyasemwal


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(executable_path='/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64')

In [5]:
driver.get("https://www.amazon.in/")

In [7]:
search_box = driver.find_element(By.XPATH,"//input[@name='field-keywords']")
search_term = input("cookware")

cookwaredosa tawa


In [8]:
search_box.send_keys(search_term)

In [9]:
search_button = driver.find_element(By.XPATH,"//input[@value='Go']")
search_button.click()
time.sleep(5)

In [10]:
search_results = driver.find_elements(By.XPATH,"//div[@class='s-result-item']")
for result in search_results:
    title = result.find_element(By.XPATH,".//h2/a/span").text
    print(title)

PRODUCT DATAFRAME AND CSV

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd

In [12]:
driver_service = Service('/Users/sukanyasemwal/Downloads/chromedriver_mac_arm64/chromedriver')
driver = webdriver.Chrome(service=driver_service)

In [13]:
df = pd.DataFrame(columns=['Brand', 'Product Name', 'Price', 'Return/Exchange', 'Expected Delivery', 'Availability', 'Product URL'])


In [14]:
def extract_product_details(result):
    # extract brand name
    try:
        brand = result.find_element(By.XPATH,".//div[@class='s-result-list s-search-results sg-row']/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]").text
    except:
        brand = "-"
        
    # extract product name
    try:
        product_name = result.find_element(By.XPATH,".//h2/a/span").text
    except:
        product_name = "-"
    
    # extract price
    try:
        price = result.find_element(By.XPATH,".//a[@class='a-link-normal a-text-normal']/span[1]/span[2]/span[2]").text
    except:
        price = "-"
    
    # extract return/exchange
    try:
        return_exchange = result.find_element(By.XPATH,".//span[contains(text(),'Return or exchange')]/parent::div/following-sibling::div/span[1]").text
    except:
        return_exchange = "-"
    
    # extract expected delivery
    try:
        expected_delivery = result.find_element(By.XPATH,".//span[contains(text(),'Arrives:')]/parent::div/span[1]").text
    except:
        expected_delivery = "-"
    
    # extract availability
    try:
        availability = result.find_element(By.XPATH,".//span[contains(text(),'in stock')]").text
    except:
        availability = "-"
    
    # extract product URL
    try:
        product_url = result.find_element(By.XPATH,".//h2/a").get_attribute('href')
    except:
        product_url = "-"
    
    # return a dictionary of extracted details
    return {
        'Brand': brand,
        'Product Name': product_name,
        'Price': price,
        'Return/Exchange': return_exchange,
        'Expected Delivery': expected_delivery,
        'Availability': availability,
        'Product URL': product_url}


In [15]:
for i in range(1, 4):
    # navigate to the search results page
    driver.get(f'https://www.amazon.in/s?k={search_term}&page={i}')
    
    # wait for the page to load
    time.sleep(5)
    
    # extract all the products listed on the page
    search_results = driver.find_elements(By.XPATH,"//div[@class='s-result-item']")
    
    # loop through the search results and extract details for each product
    for result in search_results:
        product_details = extract_product_details(result)
        
        # add the details to the DataFrame
        df = df.append(product_details, ignore_index=True)


In [16]:
df.to_csv('amazon_search_results.csv', index=False)